# Ejemplo de uso del código
Autor: Ignacio Zurita Tapia.

# Datos y github

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!wget https://data.deepai.org/miniimagenet.zip

In [ ]:
!unzip /content/miniimagenet.zip

In [ ]:
!git clone https://github.com/iazt/few-shot-learning.git

# Carga de datos


In [ ]:
import sys
sys.path.append("/content/few-shot-learning/")

In [ ]:
from train import train
from src.dataset import miniImagenet
from src.baseline import Baseline, Baseline_plus
from src.conv4 import Conv4
from src.conv6 import Conv6

In [ ]:
train_dataset = miniImagenet('/content/mini-imagenet-cache-train.pkl',augmentation=False, split= True)
val_dataset = train_dataset.get_val_dataset()
test_dataset = miniImagenet('/content/mini-imagenet-cache-test.pkl', augmentation=False, split = False)

In [ ]:
from torch.utils.data import DataLoader
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False) # datos ordenados por clase




# Entrenamiento de Backbones

Se puede crear la red Baseline o Baseline_plus, teniendo ambos como input el número de clases y el backbone respectivo.

In [ ]:
net = Baseline(64, Conv4)
net.to('cuda')

In [ ]:
epochs  = 400 #Ocupado en el paper
dir_checkpoint = '/content/gdrive/My Drive/fewshot/baseline_checkpoints/'
val_acc, val_loss, tiempo = train(net, epochs, train_loader, val_loader, name = 'new_code_noaugnosoft', dir_checkpoint=dir_checkpoint)

In [ ]:
import torch
dir_checkpoint = '/content/gdrive/My Drive/fewshot/baseline_checkpoints/'
torch.save(net.state_dict(), dir_checkpoint + 'new_code_noaug_lastmodel.pth')

# Few shot

Se cargan los pesos del mejor modelo y se evalúa la etapa de few shot con la función *few_shot*, congelando el embedding.


In [ ]:
import torch.nn as nn
import torch
from src.few_shot_episode import freeze_embedding
net = Baseline_plus(64, Conv6)
dir_checkpoint = '/content/gdrive/My Drive/fewshot/baselineplus/conv6/'
net.load_state_dict(torch.load(dir_checkpoint+ 'conv6_best_model.pth'))
net.to('cuda')
net.classifier.fc = nn.Linear(128,5, bias = False)
net.classifier.fc.to('cuda')
freeze_embedding(net)

In [ ]:
from train_fewshot import few_shot

acc, loss = few_shot(net, test_dataset, n_episodes = 600)

In [ ]:
import pandas as pd
df = pd.DataFrame({"acc":acc, "loss":loss})
df.to_csv(dir_checkpoint+'conv6baselineplus.csv', index=False)

#NED

Se ejecuta el algoritmo NED, mediante la función *eval_Ned*.

In [ ]:
import torch
from src.few_shot_episode import freeze_embedding
net = Baseline_plus(64, Conv6)
dir_checkpoint = '/content/gdrive/MyDrive/fewshot/baselineplus/conv6/'
net.load_state_dict(torch.load(dir_checkpoint+ 'conv6_best_model.pth'))
net.to('cuda')
freeze_embedding(net)

In [ ]:
from NED import ned, eval_Ned
acc = eval_Ned(net, test_dataset, n_episodes = 600)

In [ ]:
acc_item = [a.item()for a in acc]

In [ ]:
import pandas as pd
df = pd.DataFrame({"acc":acc_item})
df.to_csv(dir_checkpoint+'NEDconv6.csv', index=False)

In [ ]:
np.mean(acc_item)